In [1]:
from utils.__init__ import *
img_folder = "G:/Github/ocr_hanviet/data/test"
pdf_folder = "G:/Github/ocr_hanviet/data/pdf"
output_path = "G:/Github/ocr_hanviet/data/ocr_output.csv"

In [ ]:
# CHUYỂN TỪ PDF SANG ẢNH JPG
for tap_number in range(6, 11): 
    pdf_path = f"{pdf_folder}/tap_{tap_number}.PDF"

    try:
        doc = fitz.open(pdf_path)
    except Exception as e:
        print(f"❌ Không thể mở file {pdf_path}: {e}")
        continue

    for page_number in range(len(doc)):
        try:
            page = doc.load_page(page_number)
            pix = page.get_pixmap(dpi=300)
            output_path = f"{img_folder}/page_{tap_number}_{page_number + 1}.jpg"
            pix.save(output_path)
            print(f"✅ Đã lưu: {output_path}")
        except Exception as e:
            print(f"❌ Lỗi khi xử lý trang {page_number + 1} của tập {tap_number}: {e}")

    doc.close()


In [3]:
class GPT_OCR:
    def __init__(self, api_key: str, img_folder: str):
        self.api_key = api_key
        self.img_folder = img_folder
        self.last_section_id = None
        openai.api_key = self.api_key

    def _load_image(self, image_path: str) -> Optional[str]:
        if not os.path.exists(image_path):
            print(f"❌ File không tồn tại: {image_path}")
            return None
        try:
            with open(image_path, "rb") as f:
                return base64.b64encode(f.read()).decode("utf-8")
        except Exception as e:
            print(f"❌ Không thể đọc ảnh: {e}")
            return None

    def _call_gpt_ocr(self, image_data_url: str) -> Optional[str]:
        try:
            response = openai.ChatCompletion.create(
                model="gpt-4o",
                messages=[
                    {
                        "role": "system",
                        "content": (
                            "Bạn là trợ lý OCR chuyên nghiệp. Hãy trích xuất từng DÒNG văn bản tiếng Việt in trong ảnh. "
                            "Với mỗi dòng, trả về một dictionary có: "
                            "'text': nội dung của dòng, và 'bbox': tọa độ bounding box dạng [x0, y0, x1, y1]. "
                            "Chỉ trả về một list các dictionaries, không giải thích thêm."
                        )
                    },
                    {
                        "role": "user",
                        "content": [
                            {
                                "type": "image_url",
                                "image_url": {"url": image_data_url}
                            }
                        ]
                    }
                ],
                temperature=0,
                max_tokens=2000,
            )
            return response["choices"][0]["message"]["content"]
        except Exception as e:
            print(f"❌ Error calling GPT-4o API: {e}")
            return None

    def _parse_response(self, content: str) -> List[Dict]:
        if not content:
            return []

        content_clean = content.strip()
        if content_clean.startswith("```"):
            content_clean = re.sub(r"^```(?:json)?\s*", "", content_clean)
            content_clean = re.sub(r"\s*```$", "", content_clean)

        try:
            ocr_lines = json.loads(content_clean)
            if isinstance(ocr_lines, list):
                return ocr_lines
        except json.JSONDecodeError:
            try:
                print("⚠️ JSON lỗi, thử parse bằng ast...")
                ocr_lines = ast.literal_eval(content_clean)
                if isinstance(ocr_lines, list):
                    return ocr_lines
            except Exception as e_ast:
                print("❌ Parse thất bại:", e_ast)
                print("🔍 Content preview:", repr(content_clean[:300]))
        return []

    def _build_dataframe(self, ocr_lines: List[Dict], image_name: str, paper_name:str, file_id: str) -> pd.DataFrame:
        data = []
        current_section = None
        sentence_buffer = []
        bbox_buffer = []

        def is_sentence_ending(text: str) -> bool:
            text = text.strip()
            return (
                text.endswith((".", ":", "!", "?"))
                or text.isupper()
                or text.startswith("-")
                or len(text.split()) <= 3
            )

        for idx, line in enumerate(ocr_lines):
            if not isinstance(line, dict) or "text" not in line:
                continue

            text = line["text"].strip()
            if not text:
                continue
            bbox = line.get("bbox", [0, 0, 0, 0])
            if not isinstance(bbox, list) or len(bbox) != 4:
                bbox = [0, 0, 0, 0]

            match = re.search(r"(quyển\s+\w+)", text, re.IGNORECASE)
            if match:
                current_section = match.group(1)
                self.last_section_id = current_section  # ✅ Cập nhật section ID dùng cho ảnh tiếp theo

            sentence_buffer.append(text)
            bbox_buffer.append(bbox)

            next_line = ocr_lines[idx + 1]["text"].strip() if idx + 1 < len(ocr_lines) else ""
            ends_now = is_sentence_ending(text)
            starts_new = next_line.startswith("-") or next_line.isupper()

            if ends_now or starts_new or idx == len(ocr_lines) - 1:
                full_text = " ".join(sentence_buffer).strip()
                sentence_buffer.clear()

                sentences = [s.strip() for s in full_text.split('.') if s.strip()]
                if not sentences:
                    sentences = [full_text]

                for i, sent in enumerate(sentences):
                    full_sentence = sent if sent.endswith(('.', '!', '?', ':')) else sent + '.'
                    data.append({
                        "File_id": file_id,
                        "Paper_id": paper_name,
                        "Section_id": current_section or self.last_section_id,
                        "Page_id": image_name,
                        "BBox": bbox_buffer[0] if bbox_buffer else [0, 0, 0, 0],
                        "Text": full_text if i == 0 else "",
                        "Sentence": full_sentence
                    })
                bbox_buffer.clear()
        return pd.DataFrame(data)

    def ocr_image(self, image_path: str, image_name: str, paper_name:str, file_id: str = "HVE_009") -> pd.DataFrame:
        print(f"📤 Đang OCR: {image_name}")
        image_b64 = self._load_image(image_path)
        if not image_b64:
            return pd.DataFrame()

        content = self._call_gpt_ocr(f"data:image/jpeg;base64,{image_b64}")
        if not content:
            print("❌ Không nhận được phản hồi từ GPT")
            return pd.DataFrame()

        ocr_lines = self._parse_response(content)
        if not ocr_lines:
            print("❌ OCR lines rỗng")
            return pd.DataFrame()

        return self._build_dataframe(ocr_lines, image_name, paper_name, file_id)

    def ocr_folder_parallel(self, max_workers: int = 4, output_file: Optional[str] = None) -> pd.DataFrame:
        if not os.path.exists(self.img_folder):
            print(f"❌ Folder không tồn tại: {self.img_folder}")
            return pd.DataFrame()

        files = sorted(f for f in os.listdir(self.img_folder) if f.lower().startswith("page_") and f.lower().endswith(".jpg"))
        if not files:
            print("❌ Không tìm thấy ảnh hợp lệ")
            return pd.DataFrame()

        print(f"🖼️ Tìm thấy {len(files)} ảnh, bắt đầu OCR với {max_workers} workers...")

        results = []
        with ThreadPoolExecutor(max_workers=max_workers) as executor:
            future_to_file = {
                executor.submit(self.ocr_image, os.path.join(self.img_folder, f), f.split("_")[-1].replace(".jpg", ""), f.split("_")[1]): f
                for f in files
            }

            for i, future in enumerate(as_completed(future_to_file), 1):
                file = future_to_file[future]
                try:
                    df = future.result()
                    if not df.empty:
                        results.append(df)

                        # ✅ Ghi append vào output_file ngay sau khi có kết quả
                        if output_file:
                            write_header = not os.path.exists(output_file)
                            df.to_csv(output_file, mode='a', index=False, encoding='utf-8-sig', header=write_header)

                        print(f"✅ ({i}/{len(files)}) {file}: {len(df)} dòng")
                    else:
                        print(f"⚠️ ({i}/{len(files)}) {file}: Không có dòng")
                except Exception as e:
                    print(f"❌ ({i}/{len(files)}) {file}: {e}")

        if results:
            df_all = pd.concat(results, ignore_index=True)
            print(f"📊 Tổng cộng {len(df_all)} dòng từ {len(results)} ảnh")
            if output_file:
                try:
                    df_all.to_csv(output_file, index=False, encoding='utf-8-sig')
                    print(f"💾 Đã lưu kết quả vào: {output_file}")
                except Exception as e:
                    print(f"⚠️ Không thể lưu file: {e}")
            return df_all
        else:
            print("❌ Không có dữ liệu nào được trích xuất")
            return pd.DataFrame()


In [4]:
extractor = GPT_OCR(api_key=OPENAI_API_KEY, img_folder=img_folder)
df_all = extractor.ocr_folder_parallel(max_workers=10, output_file=output_path)


🖼️ Tìm thấy 11 ảnh, bắt đầu OCR với 10 workers...
📤 Đang OCR: 1
📤 Đang OCR: 2
📤 Đang OCR: 3
📤 Đang OCR: 4
📤 Đang OCR: 5
📤 Đang OCR: 6
📤 Đang OCR: 7
📤 Đang OCR: 8
📤 Đang OCR: 9
📤 Đang OCR: 189
📤 Đang OCR: 2
✅ (1/11) page_6_2.jpg: 8 dòng
✅ (2/11) page_6_1.jpg: 6 dòng
✅ (3/11) page_7_2.jpg: 8 dòng
✅ (4/11) page_6_3.jpg: 17 dòng
✅ (5/11) page_6_5.jpg: 18 dòng
✅ (6/11) page_6_9.jpg: 18 dòng
✅ (7/11) page_6_8.jpg: 18 dòng
✅ (8/11) page_7_189.jpg: 15 dòng
✅ (9/11) page_6_4.jpg: 16 dòng
✅ (10/11) page_6_7.jpg: 17 dòng
✅ (11/11) page_6_6.jpg: 12 dòng
📊 Tổng cộng 153 dòng từ 11 ảnh
💾 Đã lưu kết quả vào: G:/Github/ocr_hanviet/data/ocr_output.csv
